# Notebook 4.1: ChatGLM2-6B

## 4.1.1 Overview
This example shows how to run [ChatGLM2-6B](https://github.com/THUDM/ChatGLM2-6B) Chinese inference on low-cost PCs (without the need of discrete GPU) using [BigDL-LLM](https://github.com/intel-analytics/BigDL/tree/main/python/llm) APIs. ChatGLM2-6B is the second-generation version of the open-source bilingual (Chinese-English) chat model [ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B) proposed by [THUDM](https://github.com/THUDM). ChatGLM2-6B also can be found in [Huggingface models](https://huggingface.co/models) in following [link](https://huggingface.co/THUDM/chatglm2-6b).

Before conducting inference, you may need to prepare environment according to [Chapter 2](../ch_2_Environment_Setup/README.md).

## 4.1.2 Installation

First of all, install BigDL-LLM in your prepared environment. For best practices of environment setup, refer to [Chapter 2](../ch_2_Environment_Setup/README.md) in this tutorial.

In [ ]:
!pip install --pre --upgrade bigdl-llm[all]

The all option is for installing other required packages by BigDL-LLM.

## 4.1.3 Load Model and Tokenizer

### 4.1.3.1 Load Model

Load ChatGLM2 model with low-bit optimization(INT4) for lower resource cost using BigDL-LLM APIs, which convert the relevant layers in the model into INT4 format.

> **Note**
>
> BigDL-LLM has supported `AutoModel`, `AutoModelForCausalLM`, `AutoModelForSpeechSeq2Seq` and `AutoModelForSeq2SeqLM`. The AutoClasses help users automatically retrieve the relevant model, in this case, we can simply use `AutoModel` to load.

> **Note**
>
> You can specify the argument `model_path` with both Huggingface repo id or local model path.

In [ ]:
from bigdl.llm.transformers import AutoModel

model_path = "THUDM/chatglm2-6b"
model = AutoModel.from_pretrained(model_path,
                                  load_in_4bit=True,
                                  trust_remote_code=True)

### 4.1.3.2 Load Tokenizer

A tokenizer is also needed for LLM inference. It is used to encode input texts to tensors to feed to LLMs, and decode the LLM output tensors to texts. You can use [Huggingface transformers](https://huggingface.co/docs/transformers/index) API to load the tokenizer directly. It can be used seamlessly with models loaded by BigDL-LLM.

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

## 4.1.4 Inference

### 4.1.4.1 Create Prompt Template

Before generating, you need to create a prompt template. Here we give an example prompt template for question and answering refers to [ChatGLM2-6B prompt template](https://huggingface.co/THUDM/chatglm2-6b/blob/main/modeling_chatglm.py#L1007). You can tune the prompt based on your own model as well.

In [22]:
CHATGLM_V2_PROMPT_TEMPLATE = "问：{prompt}\n\n答："

### 4.1.4.2 Generate

Then, you can generate output with loaded model and tokenizer.

> **Note**
> 
> `max_new_tokens` parameter in the `generate` function defines the maximum number of tokens to predict. 

In [25]:
import torch

prompt = "AI是什么？"
n_predict = 32

with torch.inference_mode():
    prompt = CHATGLM_V2_PROMPT_TEMPLATE.format(prompt=prompt)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids,
                            max_new_tokens=n_predict)
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)
    print('-'*20, 'Output', '-'*20)
    print(output_str)

-------------------- Output --------------------
问:AI是什么?

答: AI是人工智能(Artificial Intelligence)的缩写,指的是一种能够模拟人类智能的技术或系统。AI系统可以通过学习、推理、解决问题等方式,实现类似于


### 4.1.4.3 Stream Chat

ChatGLM2-6B support streaming output function `stream_chat`, which enable the model to provide a streaming response word by word. However, other models may not provide similar APIs, if you want to implement general streaming output function, please refer to [Chapter 5.1](../ch_5_AppDev_Intermediate/5_1_ChatBot.ipynb).

> **Note**
>
> To successfully observe the text streaming behavior in standard output, we need to set the environment variable `PYTHONUNBUFFERED=1 `to ensure that the standard output streams are directly sent to the terminal without being buffered first.

In [6]:
import torch

with torch.inference_mode():
    question = "AI 是什么？"
    response_ = ""
    print('-'*20, 'Stream Chat Output', '-'*20)
    for response, history in model.stream_chat(tokenizer, question, history=[]):
        print(response.replace(response_, ""), end="")
        response_ = response

-------------------- Stream Chat Output --------------------
AI指的是人工智能,是一种能够通过学习和理解数据,以及应用适当的算法和数学模型,来执行与人类智能相似的任务的计算机程序。AI可以包括机器学习、自然语言处理、计算机视觉、专家系统、强化学习等不同类型的技术。

AI的应用领域广泛,例如自然语言处理可用于语音识别、机器翻译、情感分析等;计算机视觉可用于人脸识别、图像识别、自动驾驶等;机器学习可用于预测、分类、聚类等数据分析任务。

AI是一种非常有前途的技术,已经在许多领域产生了积极的影响,并随着技术的不断进步,将继续为我们的生活和工作带来更多的便利和改变。

## 4.1.5 Use in LangChain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a widely used framework for developing applications powered by language models. In this section, we will show how to integrate BigDL-LLM with LangChain. You can follow this [instruction](https://python.langchain.com/docs/get_started/installation) to prepare environment for LangChain.

If you need, install LangChain as following, or you can refer to [Chapter 8](../ch_8_AppDev_Advanced/README.md) for more information about LangChain integrations:

In [ ]:
!pip install -U langchain==0.0.248

> **Note**
> 
> We recommend to use `langchain==0.0.248`, which is verified in our tutorial.

### 4.1.5.1 Create Prompt Template

Before inference, you need to create a prompt template. Here we give an example prompt template for question and answering, which contains two input variables, `history` and `human_input`. You can tune the prompt based on your own model as well.

In [10]:
CHATGLM_V2_LANGCHAIN_PROMPT_TEMPLATE = """{history}\n\n问：{human_input}\n\n答："""

### 4.1.5.2 Prepare Chain

Use [LangChain API](https://api.python.langchain.com/en/latest/api_reference.html) `LLMChain` to construct a chain for inference. Here we use BigDL-LLM APIs to construct a `LLM` object, which will load model with low-bit optimization automatically.

> **Note**
>
> `ConversationBufferWindowMemory` is a type of memory in LangChain that keeps a sliding window of the most recent `k` interactions in a conversation.

In [ ]:
from langchain import LLMChain, PromptTemplate
from bigdl.llm.langchain.llms import TransformersLLM
from langchain.memory import ConversationBufferWindowMemory

llm_model_path = "THUDM/chatglm2-6b" # the path to the huggingface llm model

prompt = PromptTemplate(input_variables=["history", "human_input"], template=CHATGLM_V2_LANGCHAIN_PROMPT_TEMPLATE)
max_new_tokens = 128

llm = TransformersLLM.from_model_id(
        model_id=llm_model_path,
        model_kwargs={"trust_remote_code": True},
)

# Following code are complete the same as the use-case
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    llm_kwargs={"max_new_tokens":max_new_tokens},
    memory=ConversationBufferWindowMemory(k=2),
)


### 4.1.5.3 Generate

In [17]:
text = "AI 是什么？"
response_text = llm_chain.run(human_input=text,stop="\n\n")




> Entering new LLMChain chain...
Prompt after formatting:


问：AI 是什么？

答：
AI指的是人工智能,是一种能够通过学习和理解数据,以及应用适当的算法和数学模型,来执行与人类智能相似的任务的技术。AI可以包括机器学习、自然语言处理、计算机视觉、知识表示、推理、决策等多种技术。



> Finished chain.
